In [ ]:
!pip install -U sentence-transformers -q

In [ ]:
pip install -U "transformers>=4.57.1" "huggingface_hub>=1.01" -q

In [2]:
import pandas as pd
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

train = pd.read_csv('/kaggle/input/pan-cief/train.csv')
test = pd.read_csv('/kaggle/input/pan-cief/val.csv')

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "sentence-transformers/all-MiniLM-L6-v2",
    local_files_only=False
)
## helper function
def get_embeddings(model,samples):
    print("generating embeddings..........")
    embeddings = model.encode(samples,device=device)
    print("finished generating embeddings.......")
    return embeddings

2025-11-04 12:59:02.100060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762261142.117761     244 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762261142.122915     244 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `ty

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
train_embeddings = get_embeddings(model,train['text'])
test_embeddings = get_embeddings(model,test['text'])

generating embeddings..........


Batches:   0%|          | 0/741 [00:00<?, ?it/s]

finished generating embeddings.......
generating embeddings..........


Batches:   0%|          | 0/113 [00:00<?, ?it/s]

finished generating embeddings.......


In [16]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

results = pd.DataFrame()

xgb_model = XGBClassifier(random_state=47,eval_metric='logloss',device='cuda',tree_method='hist')
xgb_model.fit(train_embeddings,train['label'])
print('Predicting on Validation set of PAN CLEF.....')
preds = xgb_model.predict(test_embeddings)

Predicting on Validation set of PAN CLEF.....


In [19]:
print(f"The accuracy_score  is {accuracy_score(test['label'], preds)}")
print(f"The precision_score  is {precision_score(test['label'], preds)}")
print(f"The recall_score  is {recall_score(test['label'], preds)}")
print(f"The f1_score  is {f1_score(test['label'], preds)}")

The accuracy_score  is 0.861242685984954
The precision_score  is 0.8642570281124498
The recall_score  is 0.9307958477508651
The f1_score  is 0.8962932111620159


## evaluation on coling (10k)

In [20]:
coling_test = pd.read_csv('/kaggle/input/creating-with-new-features-coling/coling_test3000.csv')
coling_test_embeddings = get_embeddings(model,coling_test['text'])

generating embeddings..........


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

finished generating embeddings.......


In [22]:
colings_preds = xgb_model.predict(coling_test_embeddings)

print(f"The accuracy_score  is {accuracy_score(coling_test['label'], colings_preds)}")
print(f"The precision_score  is {precision_score(coling_test['label'], colings_preds)}")
print(f"The recall_score  is {recall_score(coling_test['label'], colings_preds)}")
print(f"The f1_score  is {f1_score(coling_test['label'], colings_preds)}")

The accuracy_score  is 0.5653
The precision_score  is 0.6663861302810975
The recall_score  is 0.6260278304870335
The f1_score  is 0.6455768446799837
